In [1]:
from modules.crawler import download
from bs4 import BeautifulSoup
import sqlite3, re
from requests.compat import urljoin

In [2]:
url = 'http://www.akomnews.com/bbs/search.php'
params = {'sfl': 'wr_subject||wr_content', 'sop': 'and', 'page':1}

In [3]:
conn = sqlite3.connect('herb.db')
cur = conn.cursor()

In [4]:
herbList = []
cur.execute('SELECT * FROM herb')
result = cur.fetchall()
for info in result:
    herb = {'no':info[0], 'name':re.sub(r'[^가-힣]', '', info[3])}
    herbList.append(herb)

In [5]:
conn.executescript('''
    CREATE TABLE news(
        news_id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
        herb_id INTEGER NOT NULL,
        url TEXT NOT NULL,
        title TEXT NOT NULL,
        date DATETIME NOT NULL default current_timestamp
    );
''')
conn.commit()

OperationalError: table news already exists

In [6]:
for herb in herbList:
    params['stx'] = herb['name']
    
    for pageNo in range(1, 20+1):
        try:
            params['page'] = pageNo
            html = download(url, params=params)
            soup = BeautifulSoup(html.text, 'lxml')

            linkList = soup.select('ul.art_list_all a')
            titleList = soup.select('ul.art_list_all h2.clamp.c2')
            dateList = soup.select('ul.art_list_all li.date')
            
            for link, title, date in zip(linkList, titleList, dateList):
                print(urljoin(url, link['href']), title.text, date.text)
                cur.execute('''
                    INSERT INTO news(herb_id, url, title, date)
                    VALUES (?, ?, ?, ?)
                ''', (herb['no'], urljoin(url, link['href']), title.text, date.text))
                conn.commit()
                
            if not linkList or not titleList:
                break
        except Exception as e:
            print(e)
            break

http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=36072 회원투표요구서 접수 '무산' 2019-08-13 15:20
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=6275 "실손보험 청구간소화 법안 통과 시급하다!" 2019-07-11 14:50
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=28617 ‘최종안을 한의사가 결정할 수 있다’ 진실을 말씀드립니다. 2019-06-14 10:54
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=28588 수험표에 시험문제 적었다가 전문의 불합격…법원 "정당" 2019-06-10 10:40
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=28580 “건보 수진자 수 감소와 자보 점유율 급증…결국 보장성 여부의 차이” 2019-06-07 09:00
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=5935 낙태죄의 헌법불합치 결정 "환영한다" 2019-04-12 10:18
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=28219 1형 당뇨병 환자 대상 혈당정보 DB 구축에 '반대' 2019-03-05 11:46
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=28215 한의학 교육의 현재와 미래 ⑨ 2019-03-05 09:00
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=28164 “앞으로도 모든 한의사들과 양질의 정보 나누는데 힘쓸 것” 2019-02-18 14:10
http://www.akomnews.com/bbs/

http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=17795 “2013년 수가계약 부속합의 폐기하자” 82.58% 2014-05-30 18:54
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=17726 “국가 긴급 재난시 한의약 응급의료 역할 큰 힘 될 것” 2014-05-30 11:05
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=17594 진료비 지불제도 관련 전 회원 여론조사 2014-05-23 11:01
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=17573 진료비 지불제도 관련 전 회원 여론조사 2014-05-22 09:55
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=17424 2013년 수가계약 체결시 ‘부속합의서’ 이행 여부 여론 조사 2014-05-16 10:28
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=30003 임은지 대표이사 2014-03-17 11:16
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=9574 원광대 익산·산본한방병원 폐원 결정, 무엇을 위한 것인가? 2014-02-14 10:49
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=31897 醫史學으로 읽는 近現代 韓醫學 (269) 2013-12-30 11:31
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=14727 한약제제 조제료 및 복약지도료 현실화 필요 2013-11-29 10:29
http://www.akomnews.com/bbs/board.php?bo_t

http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=13687 한약제제 489개 품목 기준 개정 2007-06-08 10:33
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=31364 “수상금은 후학 발전 위해 기부” 2007-04-20 11:13
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=13494 한약재 안전성 확보 “해법을 찾아라” 2006-12-27 08:51
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=28907 고객은 서비스를 원한다 ② 2006-12-12 09:07
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=28901 글로벌시대 한의사 학술활동을 위한 제언 ⑤ 2006-11-22 10:06
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=28831 의료시장개방과 한의계의 대책(교육 분야) 2005-12-23 13:23
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=11375 “저비용·고효율 청구방식 시스템 정착해야” 2005-10-05 12:55
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=30183 “한의학 미래위한 준비에 박차” 2005-07-01 09:07
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=13141 한약재 불법 판매 행위 집중 단속 2005-01-21 09:16
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=267 “회관 마무리 작업에 전력 다할 터” 2005-0

http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=16534 (주)글로벌허브 한방과학화연구소, 철저한 품질 검사로 한약재 신뢰 확보 2012-01-06 14:02
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=14596 11개 한약재 수급조절품목 수입 결정 2011-12-23 13:15
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=31000 허담 대표이사 2011-11-18 11:33
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=14580 여성 성기능 장애 대한 한의학적 접근법 모색 2011-11-04 13:45
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=1883 식·약 공용한약재 중금속 기준 일원화 2011-09-06 13:21
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=31690 醫史學으로 읽는 近現代 韓醫學 153 2011-06-03 09:35
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=31683 엄경섭 회장 2011-05-04 17:15
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=16241 국산 한약재의 원활한 유통에 앞장 2011-04-01 13:43
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=16208 ‘전국약용작물품목별대표총연합회’ 설립 2011-03-11 13:02
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=35640 한약재 가격 폭등 근본적인 대책이 필요 201

http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=36083 2018년 한의약 건강증진사업 사례 9 2019-08-14 15:38
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=36047 "추나요법은 과학적으로 응용·개발·발전된 현대 한의학" 2019-08-09 12:04
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=36007 경락경혈학회, 오는 24일 하계학술대회 개최 2019-08-06 09:57
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=35979 비만하면 폐 기능도 감소 2019-08-02 09:16
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=35932 “발달장애자녀 부모님, 한의약으로 돌봐드려요” 2019-07-30 14:26
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=35868 간단한 뇌파 측정으로 치매 위험군 선별 가능 2019-07-25 09:47
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=35813 2018년 한의약 건강증진사업 사례 6 2019-07-19 09:06
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=28754 사암침법학회, '사암도인 침술원리 하계캠프' 개최 2019-07-18 09:45
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=28733 2018년 한의약 건강증진사업 사례 5 2019-07-12 09:06
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_i

http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=5333 세종시, 갱년기 여성 한의약 건강증진 프로그램 운영 2018-09-27 17:13
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=12667 지난해 경도인지장애 진료인원 18만6천명…연평균 24.2% 증가 2018-09-20 15:52
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=27559 9·21 ‘세계 알츠하이머의 날’, “치매와의 전쟁에서 한의요법 적용해야” 2018-09-14 13:24
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=12659 MRI, 손팔 이식술 건보 적용 등 보장성 확대 '중점 추진' 2018-09-14 10:08
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=5293 홍삼이나 프로바이오틱스가 만병통치약? 2018-09-13 09:39
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=27524 노인 건강도 한의학으로 든든하게 2018-09-06 14:32
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=27506 평생 중력장의 영향을 받는 척추와 골격, 이상 생기면 공간척추도인안교로 치료 2018-09-03 10:33
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=5237 대한한의사협회, 제2회관 건축위원회 첫 가동 2018-08-31 15:53
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=5232 한의약으로 갱년기 극복해 ‘한번 더 청춘하자 go’ 2018-08-30 15:

http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=4083 치매 치료가 양의사의 전유물?…국민의 진료 선택권 박탈 2017-08-25 17:41
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=4076 與野 "국가적 치매 치료에서 한의약 역할 기대" 2017-08-23 18:28
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=4040 노인외래 정액제 개편시 한의계 제외되면 '총궐기 투쟁' 나설 것! 2017-08-14 16:57
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=10164 "어르신들에게 치매의 위험성 알기 쉽게 전달하고 싶었어요" 2017-07-21 11:19
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=10159 "한의치매 예방 프로그램으로 시원하게 떠올리세요!" 2017-07-17 15:24
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=15155 허리디스크, 재발없이 한의 치료로 잡는다 2017-07-17 11:07
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=12456 한의자동차보험 환자 지속적 증가 예상…제도 개선 뒷받침 '필요' 2017-07-14 11:32
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=26304 여한의사 증가, 여성의료인의 사회적 인식 개선에 영향 2017-06-30 10:52
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=15143 진피 추출물, 암에 의한 근육·체중 감소 완화효과 '규명' 2017-06-30 1

http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=28037 한의학에 딥러닝 적용 가능성 확인…약재 감별의 안전성과 효율성 검증 2019-01-11 14:19
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=27152 꼬리명주나비의 꿈(上) 2018-06-08 12:17
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=26782 ‘세계 신장의 날’을 맞이하여 2018-03-09 11:54
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=26414 의약품 '한약' 안전성 확인 2017-09-18 18:26
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=3730 한의협의 불법 한약재 모니터링…가시적 성과 나타나 2017-04-24 13:56
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=3497 국회 “식품용 한약재에 대한 관리 강화 필요하다” 2017-02-03 09:55
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=14924 한의協 "한약 임상시험, 이미 하고 있다" 2016-05-02 11:06
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=2591 식약처장, 정치권 진출을 위한 교두보인가? 2016-03-15 10:15
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=14860 식약처의 한약재 관리감독 부실 ‘여전’ 2016-02-15 16:38
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=6237 건

http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=4428 상한론에서의 복용량 단위 1승=65ml, 1양=6.5g 2017-12-22 11:53
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=26524 論으로 풀어보는 한국 한의학 (125) 2017-11-24 12:03
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=15196 대구 약령시장 한약재는 잔류농약에 '안전' 2017-10-12 15:43
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=26426 醫史學으로 읽는 近現代韓醫學 (366) 2017-09-29 09:28
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=26423 한의 없는 일본의료, 정부는 ‘반면교사’ 삼아야 2017-09-25 11:33
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=26239 한의원 세무 칼럼 – 087 2017-05-29 10:31
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=26193 論으로 풀어보는 한국 한의학 (110) 2017-04-17 11:48
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=26190 동의보감에 ‘귀신보는 방법’이 있다? 2017-04-10 14:53
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=26108 한의원 세무 칼럼 – 075 2017-02-20 10:15
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=25950 펑스룬馮世綸 

http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=6860 김동환 원장 등 원로 한의사 5명 황금살구나무한의원상 수상 2006-01-31 07:39
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=30221 강북구한의사회  이경성 회장 2006-01-20 12:34
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=28821  新 본초기행    6 2005-11-01 12:03
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=28817  新  본 초 기 행  5 2005-10-05 13:03
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=13368 의약품 수입 역조현상 ‘고착화’ 2005-08-05 09:53
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=30187 대한전통의학회 강진춘 회장 2005-07-12 09:35
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=31302 몸짱아줌마 정다연 2005-03-22 10:07
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=29172 한의사 ‘황지혜’의 인턴수련 일기(27) 2005-02-15 11:22
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=13141 한약재 불법 판매 행위 집중 단속 2005-01-21 09:16
http://www.akomnews.com/bbs/board.php?bo_table=news&wr_id=30118 섹션화로  변신 중인 ‘월간 의림’ 2004-12-10 10:44
http://ww